In [223]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from pprintpp import pprint as pp
import requests
from lxml import html

In [224]:
client = MongoClient()
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [225]:
db = client.parser_goods

In [226]:
db.list_collection_names() #проверяем коллекции

['castorama_ru']

In [227]:
print(client.list_database_names()) #проверяем базы данных

['admin', 'config', 'local', 'parser_books', 'parser_goods', 'vacancy']


In [228]:
collection = db.mvideo

In [229]:
url = 'https://mvideo.ru/'

In [230]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

In [231]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.get(url)

In [232]:
link_list = []
# button = driver.find_element(By.XPATH, '//button[@class="mv-promotions-carousel--control-button mv-promotions-carousel--control-button-right mv-icon-button--primary mv-icon-button--medium mv-icon-button--shadow mv-button mv-icon-button ng-star-inserted"]')
# button.click()
new_pages = driver.find_elements(By.XPATH, '//a[@_ngcontent-serverapp-c206]')
for page in new_pages:
    page_link = page.get_attribute("href")
    link_list.append(page_link)
    sleep(1)

In [233]:
link_list

['https://www.mvideo.ru/promo/promocatalog?from=hb',
 'https://www.mvideo.ru/promo/akciya-rassrochka-mark199208912?from=hb',
 'https://www.mvideo.ru/promo/nedeli-realme?from=hb',
 'https://www.mvideo.ru/promo/skidki-do-40-na-tehniku-rowenta-ultimate-mark202684788?from=hb',
 'https://www.mvideo.ru/promo/rassrochka-na-vse-iphone?from=hb',
 'https://www.mvideo.ru/promo/vygoda-na-noutbuki-huawei?from=hb',
 'https://www.mvideo.ru/promo/smart-chasy-xiaomi-smart-band-7-pro-v-podarok-pri-pokupke-smartfona-xiaomi-13-lite?from=hb',
 'https://www.mvideo.ru/promo/besplatnaya-ekspress-dostavka-pri-pokupke-ot-4-990-r-mark202300238?from=hb',
 'https://www.mvideo.ru/promo/umnaya-pererabotka-tehniki?from=hb']

In [234]:
for link in link_list:
    driver.get(link)
    elements = driver.find_elements(By.XPATH, '//img')
    sleep(1)
    for el in elements:
        photo = {'photo': el.get_attribute('src')}
        photo_in_db = collection.find(photo)
        photo_in_db_list = []
        for el in photo_in_db:
            photo_in_db_list.append(el)
        if photo_in_db_list == []:
            collection.insert_one(photo)
        sleep(1)

In [235]:
count_docs = collection.find()
num = 0
for i in count_docs:
    num += 1
print(f'В базе данных "parser_goods" в коллекции mvideo после внесения новых товаров всего {num} товаров.') 

В базе данных "parser_goods" в коллекции mvideo после внесения новых товаров всего 72 товаров.
